# 提取所有题目信息

In [4]:
import json
import pandas as pd

In [6]:
LANGUAGE_LIST = ["c&cpp"]
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}


def generate_python_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a python function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a python class , the class signature as below {code_signature}"


def generate_javascript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a javascript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a javascript class , the class signature as below {code_signature}"


def generate_typescript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a typescript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a typescript class , the class signature as below {code_signature}"


def generate_ccpp_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a cpp function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a cpp class , the class signature as below {code_signature}"


LANGUAGE_PROMPT_MAP = {
    "python": generate_python_prompt,
    "javascript": generate_javascript_prompt,
    "typescript": generate_typescript_prompt,
    "c&cpp": generate_ccpp_prompt
}

In [7]:
for language in LANGUAGE_LIST:
    # 读取excel表 获取数据信息
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    question_array = []
    for index, row in data.iterrows():
        task_id = int(row["task_id"])
        code_type = row["code_type"]
        dir_path = f"../all/t{task_id}"
        signature_path = f"{dir_path}/{language}/signature.{SUFFIX_MAP[language]}"
        addition_info = str(row["addition_info"])
        if addition_info == 'nan':
            addition_info = ""
        with open(signature_path, "r", encoding="utf8") as signature_file:
            code_signature = signature_file.read()
        test_path = f"{dir_path}/{language}/test.{SUFFIX_MAP[language]}"
        with open(test_path, "r", encoding="utf8") as test_file:
            code_test = test_file.read()
        prompt = LANGUAGE_PROMPT_MAP[language](code_signature, code_type)
        question_info = {
            "task_id": task_id,
            "code_type": code_type,
            "code_signature": code_signature,
            "code_language": language,
            "test_code": code_test,
            "prompt": prompt,
            "addition_info": addition_info

        }
        question_array.append(question_info)
        print(f"task_id:{task_id} signature:{len(code_signature)} test:{len(code_test)}")
    with open(f"./question/{language}.json", "w", encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()
    with open(rf"E:\code\code_back\python_project\llm\qa\question\{language}.json","w",encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()

task_id:183 signature:1046 test:1843
task_id:184 signature:1089 test:1467
task_id:187 signature:432 test:1302
task_id:188 signature:124 test:1100
task_id:189 signature:429 test:1129
task_id:190 signature:201 test:1173
task_id:191 signature:360 test:923
task_id:192 signature:219 test:1153
task_id:193 signature:333 test:452
task_id:194 signature:316 test:1015
task_id:195 signature:383 test:1191
task_id:196 signature:390 test:776
task_id:197 signature:640 test:553
task_id:198 signature:283 test:1058
task_id:200 signature:361 test:1433
task_id:202 signature:648 test:872
task_id:203 signature:329 test:1192
task_id:204 signature:395 test:1140
task_id:205 signature:212 test:1498
task_id:206 signature:639 test:845
task_id:207 signature:411 test:1337
task_id:208 signature:718 test:2695
task_id:209 signature:740 test:2299
task_id:210 signature:143 test:777
task_id:354 signature:609 test:1952
task_id:355 signature:617 test:1857
task_id:356 signature:167 test:879
task_id:357 signature:214 test:142